In [ ]:
## BASH script for checking the files 

# List of all the files: 
find . -name *CO4* -maxdepth 3 -mindepth 3 > /nfs/research/birney/users/fanny/medaka/ziram_analysis/list_files_01062023.txt

# Count number of files per plate / per line (can help to see which are missing):
find . -name *CO4* -maxdepth 3 -mindepth 3 |cut -d/ -f2 | sort  | uniq -c | tee /nfs/research/birney/users/fanny/medaka/Ziram_analysis/number_files_per_plates_01062023.txt
find . -name *CO4* -maxdepth 3 -mindepth 3 |cut -d/ -f3 | cut -d ' ' -f1 | sort  | uniq -c | tee /nfs/research/birney/users/fanny/medaka/Ziram_analysis/number_files_per_line_01062023.txt

# Modification of files encoding (not workinng on the cluster because miss library)
find . -name *.tif -exec ./ffmpeg -i {} -pix_fmt rgb8 -y {}.png \;
for i in test_set/*.tif; do ./ffmpeg -i "$i" -pix_fmt pal8 -y "${i%.*}.png"; done

#  

In [11]:
import glob
import os
import shutil
import sys
import pandas as pd
import re


In [ ]:

files = pd.read_table(file, header=None)
files[['plate', 'well']] = files[0].str.split('/', expand=True)[[0, 2]]
files['well'] = files['well'].str.split('---', n=1, expand=True)[0].str.replace('WE00', '').astype(int).astype(str)
files['plate'] = files['plate'].str.split('_', n=1, expand=True)[0].astype(str)
files['plate_well'] = files['plate'] + '_' + files['well'].astype(int).astype(str)
files


In [151]:
metadata = pd.read_excel('Germany Data Organized.xlsx', engine='openpyxl')
print(metadata.shape)
metadata.dropna(subset=['Plate #', 'Well #', '# kinks'], inplace=True)
metadata.dropna(how='all', axis=1, inplace=True)
# metadata.dropna(inplace=True)
print(metadata.shape)
metadata['plate_well'] = metadata['Plate #'] + '_' + metadata['Well #'].astype(int).astype(str)
metadata.columns = [col.split('\n')[0].replace(' ', '_').replace('?', '').lower() for col in metadata.columns]
metadata.head()
# metadata['unnamed:_11'].astype(str).value_counts()

(1726, 12)
(932, 11)


,start_exposure,line,exposure_type,stop_exposure,fixed,imaged,plate_#,well_#,kinked,#_kinks,severity_score,plate_well
0,2022-07-20,117-1,Ziram,2022-07-30,2022-08-04,2022-08-18 00:00:00,Plate 8,13.0,1,10+,4.0,Plate 8_13
1,2022-07-20,117-1,Ziram,2022-07-30,2022-08-04,2022-08-18 00:00:00,Plate 8,14.0,1,10+,4.0,Plate 8_14
2,2022-07-20,117-1,Ziram,2022-07-30,2022-08-04,2022-08-18 00:00:00,Plate 8,15.0,1,10+,4.0,Plate 8_15
5,2022-07-20,117-1,DMSO,2022-07-30,2022-08-04,2022-08-22 00:00:00,Plate 10,37.0,0,0,0.0,Plate 10_37
6,2022-07-20,117-1,DMSO,2022-07-30,2022-08-04,2022-08-22 00:00:00,Plate 10,49.0,0,0,0.0,Plate 10_49


In [153]:
import glob 

fil = pd.read_table('ftp_files.txt', header=None)[0].str.split('/', expand=True)
fil[['line', 'date']] = fil[2].str.split(' \(', expand=True)
fil['date'] = fil['date'].str.replace(')', '').str.split(' ', expand=True)[0]
fil['line'] = fil['line'].str.split(' ', expand=True)[0]
fil['plate'] = fil[1].str.split('_', expand=True, n=1)[0]
fil[['well', 'channel']] = fil[3].str.split('--', expand=True)[[0, 4]]
fil[3] = fil[3].str.split('\.', expand=True)[0]
fil = fil.groupby(['line', 'date', 'well', 'plate', 'channel'])[3].first().unstack().reset_index()
fil['plate_well'] = fil['plate'] + '_' + fil['well'].str.replace('WE0', '').astype(int).astype(str)
fil.shape

(1091, 7)

In [155]:
merge = pd.merge(fil, metadata, on='plate_well', how='outer')

print('na', merge[merge['kinked'].isna()].shape)
print('Merge shape', merge.shape)
for col in merge.filter(regex='.*_x').columns:
    print(merge[merge[col] != merge[col.replace('x', 'y')]].filter(regex=col[:-2]).value_counts())

merge[merge.duplicated(keep=False, subset='plate_well')]


na (157, 18)
Merge shape (1091, 18)
line_x  line_y
23-2    22-1      2
dtype: int64


,line_x,date,well,plate,CO4,CO6,plate_well,start_exposure,line_y,exposure_type,stop_exposure,fixed,imaged,plate_#,well_#,kinked,#_kinks,severity_score
423,22-1,25.07,WE00040,Plate 4,WE00040---D009--PO01--LO001--CO4--SL001--PX325...,WE00040---D009--PO01--LO001--CO6--SL001--PX325...,Plate 4_40,2022-07-25,22-1,Ziram,2022-08-03,2022-08-08,2022-08-17 00:00:00,Plate 4,40.0,1,10+,4.0
424,23-2,25.07,WE00040,Plate 4,WE00040---D009--PO01--LO001--CO4--SL001--PX325...,WE00040---D009--PO01--LO001--CO6--SL001--PX325...,Plate 4_40,2022-07-25,22-1,Ziram,2022-08-03,2022-08-08,2022-08-17 00:00:00,Plate 4,40.0,1,10+,4.0
427,22-1,25.07,WE00043,Plate 4,WE00043---D006--PO01--LO001--CO4--SL001--PX325...,WE00043---D006--PO01--LO001--CO6--SL001--PX325...,Plate 4_43,2022-07-25,22-1,Ziram,2022-08-03,2022-08-08,2022-08-17 00:00:00,Plate 4,43.0,1,10,4.0
428,23-2,25.07,WE00043,Plate 4,WE00043---D006--PO01--LO001--CO4--SL001--PX325...,WE00043---D006--PO01--LO001--CO6--SL001--PX325...,Plate 4_43,2022-07-25,22-1,Ziram,2022-08-03,2022-08-08,2022-08-17 00:00:00,Plate 4,43.0,1,10,4.0
429,22-1,25.07,WE00044,Plate 4,WE00044---D005--PO01--LO001--CO4--SL001--PX325...,WE00044---D005--PO01--LO001--CO6--SL001--PX325...,Plate 4_44,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
430,23-2,25.07,WE00044,Plate 4,WE00044---D005--PO01--LO001--CO4--SL001--PX325...,WE00044---D005--PO01--LO001--CO6--SL001--PX325...,Plate 4_44,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
431,22-1,25.07,WE00047,Plate 4,WE00047---D002--PO01--LO001--CO4--SL001--PX325...,WE00047---D002--PO01--LO001--CO6--SL001--PX325...,Plate 4_47,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
432,23-2,25.07,WE00047,Plate 4,WE00047---D002--PO01--LO001--CO4--SL001--PX325...,WE00047---D002--PO01--LO001--CO6--SL001--PX325...,Plate 4_47,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
433,22-1,25.07,WE00048,Plate 4,WE00048---D001--PO01--LO001--CO4--SL001--PX325...,WE00048---D001--PO01--LO001--CO6--SL001--PX325...,Plate 4_48,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
434,23-2,25.07,WE00048,Plate 4,WE00048---D001--PO01--LO001--CO4--SL001--PX325...,WE00048---D001--PO01--LO001--CO6--SL001--PX325...,Plate 4_48,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:

#verif 


# To identify if any is missing
fil[fil['plate_well'].isin(merge['plate_well']) == False]
metadata[metadata['plate_well'].isin(merge['plate_well'])== False]

## NB: Still missing one plate --> reload plate 11!

# Check for duplicates 
merge[merge['plate_well'].duplicated(keep=False)].shape
# merge[merge['plate_well'].duplicated(keep=False)].to_csv('Double_imaged_individuals_01062023.csv')


(4, 18)

In [94]:
# merge[merge['plate_well'].duplicated(keep='first') == False].to_csv('Germany_data_plate_image.csv', index=False)
merge.to_csv('Germany_Data_Organized_metadata.csv')


In [112]:
import os 
import glob

file_co6 = [os.path.basename(file).split('.')[0] for file in glob.glob('dataset_ziram_CO6/*/*')]
file_co4 = [os.path.basename(file).split('.')[0] for file in glob.glob('dataset_ziram_CO4/*/*')]

print(len(file_co6), len(file_co4))
# print(merge['CO6'].unique())

print(merge[merge['CO6'].isin(file_co6)==False].shape, merge[merge['CO4'].isin(file_co4)==False].shape)
miss_co6 = merge[merge['CO6'].isin(file_co6)==False]['plate_well'].tolist()
miss_co4 = merge[merge['CO4'].isin(file_co4)==False]['plate_well'].tolist()

# print(len(miss_co6), len(miss_co4))
# set(miss_co4) - set(miss_co6)

# print(
# merge[merge['plate_well'].duplicated(keep=False)].shape,
# merge[merge['CO4'].duplicated(keep=False)].shape
# )
# set(co4_ls) - set(co6_ls)

merge['plate_well'].nunique()

1148 1147
(0, 18) (1, 18)


932

In [149]:
df6 = pd.DataFrame(glob.glob('dataset_ziram_CO6/*/*'))[0].str.split('/', expand=True)
df4 = pd.DataFrame(glob.glob('dataset_ziram_CO4/*/*'))[0].str.split('/', expand=True)
df6[2] = df6[2].str.split('.',expand=True)[0]
df4[2] = df4[2].str.split('.',expand=True)[0]

rep = pd.read_csv('dataset_ziram_CO4/Repartition_images_sets.csv')
#CO6 test set

rep[rep['severity_score'].isna()].shape

test6 = df6[df6[1] == 'test_set'][2].tolist()
test4 = df4[df4[1] == 'test_set'][2].tolist()
train6 = df6[df6[1] == 'training_set'][2].tolist()
train4 = df4[df4[1] == 'training_set'][2].tolist()

rep[rep['CO4'].isin(train4)].shape, len(train4), 
len(set(rep['CO6']) - set(train6))
len(set(train6) & set(rep['CO6']) )

749

In [ ]:
bsub -oo CO4.out -M1G 'python3 data_split.py "dataset_cnn_CO4/*" Germany_Data_Organized_metadata.csv severity_score -p dataset_ziram_CO4 -n CO4' 

In [101]:
merge.rename(columns={merge.columns[12]: 'kinked'}, inplace=True)
merge['kinked'] = merge['kinked'].replace('1?', None, regex=False)
merge['kinked'].value_counts(normalize=True)

1    0.677237
0    0.322763
Name: kinked, dtype: float64

In [102]:
# check for the datasplit
dataset_path = 'BLOU'
column_sep = 'kinked'
ratio_test = 0.2

dict_folder = {'test':os.path.join(dataset_path, 'test_set'),
                'training': os.path.join(dataset_path, 'training_set')}

df = merge[merge['plate_well'].duplicated() == False].dropna()

# df['image'] = df['image'].str.rsplit('/', n=1, expand=True)[1]

assert df[0].value_counts().max() == 1, 'Problem with naming images, repetition or missed value !'
# df.rename(columns={'Plate (Imaged)':'plate', 'Well #': 'well', '# kinks': 'kinks', 'Severity Score:': 'score'}, inplace=True)
# df.loc[df['Exposure'] == 'DMSO', 'treated'] = 'Not_treated'
# df.loc[df['Exposure'] != 'DMSO', 'treated'] = 'Treated'
image_random_ls = []
for val, n_val in dict(df[column_sep].value_counts()).items():
    print(val, int(n_val * ratio_test))
    image_random_ls  += df[df[ column_sep].astype(str) == str(val)].sample(int(n_val * ratio_test))[0].tolist()
    dict_folder[f'training_{str(val)}'] = os.path.join(dataset_path, 'training_set', str(val))
    dict_folder[f'test_{str(val)}'] = os.path.join(dataset_path, 'test_set', str(val))



for n, file in enumerate(df[0].unique()):
    image = file.split('/')[-1]
    try:
        val = df.loc[(df[0] == file), column_sep].tolist()[0]
    except:
        print(f'ERROR(file {n}): \n',  df.loc[(df[0] == image)])
    # print(image, df.loc[(df['image']== image), 'image'].nunique(), val)
    if file in image_random_ls:
        destination_folder = f'test_{str(val)}'
    else:
        destination_folder = f'training_{str(val)}'
    
    # move_file(image, destination_folder)
    df.loc[(df[0].str.contains(image)), 'folder'] = dict_folder[destination_folder]


    

1 114
0 56


In [103]:
blou = df['folder'].str.split('/', expand=True)

# Check the repartition of the files
blou[blou[1].str.contains('training')].value_counts(normalize=True)
blou[blou[1].str.contains('test')].value_counts(normalize=True)
blou[2].value_counts(normalize=True)
blou[1].value_counts(normalize=True)

training_set    0.800469
test_set        0.199531
Name: 1, dtype: float64

In [105]:
df['image'] = df[0].str.split('/', expand=True)[3]
df.drop_duplicates(subset='plate_well', inplace=True)
df.dropna(inplace=True)
df.drop('folder', axis=1).to_csv('Germany_data_plate_image.csv', index=False)


Test for repartition accross the sets

In [7]:
df = pd.read_csv('dataset_cnn/Repartition_images_sets.csv')
print('Total repartition', df['score'].value_counts(normalize=True))
print('\nTrain\n', df[df['folder'] == 'training_set']['score'].value_counts(normalize=True))
print('\nTest\n', df[df['folder'] == 'test_set']['score'].value_counts(normalize=True))

0.0    0.348073
4.0    0.234694
3.0    0.160998
2.0    0.155329
1.0    0.100907
Name: score, dtype: float64

### Table with all files according to channel

In [14]:
df = pd.read_table('/homes/fanny/medaka/ziram_analysis/all_files_08062023.txt', header=None)[0].str.split('/', expand=True)
df['channel'] = df[3].str.split('--', expand=True)[4]
df['plate'] = df[1].str.split('_', n=1, expand=True)[0]
df['well'] = df[3].str.split('---', n=1, expand=True)[0].str.replace('WE000', '')
df.groupby(['plate', 'well', 'channel']).first()[3].unstack().reset_index().to_csv('channel_table.csv', index=False)

In [3]:
df = pd.read_csv('/homes/fanny/medaka/ziram_analysis/channel_table.csv')
sets = pd.read_csv('/homes/fanny/medaka/ziram_analysis/dataset_cnn_CO4/Repartition_images_sets.csv')
df['plate_well'] = df['plate'] + '_' + df['well'].astype(str)
merge = pd.merge(sets.drop(['filename', 'Well', 'Plate', ], axis=1), df, on=["plate_well", 'plate', 'well'])
merge.to_csv('Plate_well_repartition_all_chanels.csv', index=False)
# df.rename(columns={})

### Check datasets

In [20]:
import glob
df = pd.read_csv('/homes/fanny/medaka/ziram_analysis/Plate_well_repartition_all_chanels.csv')

df = df[df['folder'] == 'training_set']

co6 = [im.split('.')[0].split('/')[2] for im in glob.glob('dataset_ziram_CO6/training_set/*png')]
co4 = [im.split('.')[0].split('/')[2] for im in glob.glob('dataset_ziram_CO4/training_set/*png')]

list_image = []
for image in co6:
    if image not in df['CO6'].str.split('.', expand=True)[0].tolist():
        list_image.append(image)
ser = pd.Series(list_image).to_csv('spare image' , header=False, index=False)


# df = df[(df['CO6'].str.split('.', expand=True)[0].isin(co6)) & (df['CO4'].str.split('.', expand=True)[0].isin(co4))]
# print(df[].shape)
# co6[:10]
co6[:5]

['WE00082---G010--PO01--LO001--CO6--SL001--PX32500--PW0100--IN0020--TM283--X095544--Y064992--Z210481--T2692481699',
 'WE00002---A002--PO01--LO001--CO6--SL001--PX32500--PW0100--IN0020--TM283--X023472--Y011163--Z213937--T2349215044',
 'WE00022---B003--PO01--LO001--CO6--SL001--PX32500--PW0100--IN0020--TM278--X032481--Y020134--Z214475--T2851745436',
 'WE00028---C004--PO01--LO001--CO6--SL001--PX32500--PW0100--IN0020--TM280--X041490--Y029106--Z214911--T2271373564',
 'WE00018---B007--PO01--LO001--CO6--SL001--PX32500--PW0100--IN0020--TM280--X068517--Y020134--Z210593--T2253443781']

In [4]:
df['CO6'] = df['CO6'].astype(str)
df['CO4'] = df['CO4'].astype(str)

In [12]:
df.groupby('plate')[['CO4', 'CO6']].nunique(dropna=False).sort_values('CO6')

,CO4,CO6
plate,,
Plate 3,8,8
Plate 15,10,10
Plate 11,12,12
Plate 5,14,14
Plate 1,17,17
Plate 4,22,22
Plate 17,33,33
Plate 10,69,69
Plate 9,70,70
